# Сначала загрузим petals, датасет GSM8K, и импортируем необходимые модули.

:


In [5]:
from google.colab import drive
drive.mount('/content/drive',  force_remount=True)

Mounted at /content/drive


In [6]:
!pip install -q petals

In [2]:
!git clone https://github.com/openai/grade-school-math


Cloning into 'grade-school-math'...
remote: Enumerating objects: 36, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 36 (delta 10), reused 7 (delta 7), pack-reused 16
Unpacking objects: 100% (36/36), 3.13 MiB | 4.50 MiB/s, done.


In [1]:
from processer import DatasetProcesser, OutputProcesser
from utils import compute_accuracy, compute_delta
from model import Model
import random
import json
import os
import numpy as np

# Подгружаем датасет
Перемешиваем тренировочные и тестовый датасет, и удаляем из ответов конструкции типа <<5+5=10>>

In [3]:
dataset_processer = DatasetProcesser()
output_processer = OutputProcesser(mode="last_num")

dataset_processer.read_dataset_jsonl("grade-school-math/grade_school_math/data/test.jsonl")
dataset_processer.read_dataset_jsonl("grade-school-math/grade_school_math/data/train.jsonl")

dataset_processer.shuffle_dataset()


# Эксперименты с моделью BLOOM-7b при стандартном режиме CoT Prompting;
Пробегаемся по нескольким вариантам генерации, и повторяем эксперимент 2 раза, для того, чтобы избевиться от элемента стохастичности. Берем только первые 100 примеров из датасета. В конкструкции кода модели (model) в функции генерации присутствует try - поскольку иногда модель выдавала низкоуровневые ошибки, что мешало генерировать данные. По этому причине простой остановки кодя ячейки будет мало - следует перезапустить среду выполнения.

In [7]:
!mkdir results

In [ ]:


NUM_OF_LAUNCHES = 100

kwargs = {"model_size": "mini", "is_bonus": False, "gpu": True}


for gen_regime in ["sample", "top-k", "top-p"]:

    for seed in range(1,3):
        
        kwargs["gen_regime"] = gen_regime
        kwargs["seed"] = seed

        model = Model(**kwargs )

        results = []
        for i in range(NUM_OF_LAUNCHES):
            
            example = dataset_processer.dataset[i]
            prompt_index = random.sample(list(range(8)), 1)[0]
            prompt = model.prompts[prompt_index]
            prompt_plus_question = model.prompt_plus_quesion(prompt, example["question"] )
            outputs = model.produce_answer(prompt_plus_question)
            result = {}
            result["indx_example"] = i
            result["prompt_index"] = prompt_index
            result["outputs"] = outputs
            
            print(i, outputs, output_processer.parse_num(outputs))

            results.append(result)
            
            path_for_results = "results/results_{0}_{1}_{2}_seed_{3}.jsonl".format(kwargs["model_size"], "standard", gen_regime, seed)
            with open(path_for_results, 'w') as f:
                json.dump(results, f)
            
            os.system("cp {} /content/drive/MyDrive".format(path_for_results))




0  Each balloon holds 10 liters of air. So if he bought 500-balloon tanks of gas, he had 500*10 = 5000 liters of air. So his total gas needed is 5000 + (1000 / 500) = 200 tanks of gas.
 200.0
1  Mark has 12 candy bars. He has 3 Snickers and 2 Mars bars. That makes 11 Mars bars and 1 Snickers. 12 – 11 = 1. The answer is 1.
 1.0
2  The first number is the quarters. 32 quarter + 95 dime + 120 nickel + 750 penny =  $ 1000.
 1000.0
3  In his task, he will have 5 x 10 x 7 = 1250 teeth. So far he has cleaned 900 teeth. So he is going to clean 850 teeth today.
 850.0
4  We know that she needs 5000 * 30 = 15000 feet of wire mesh. The wire mesh that was left over will be $13200 which is not sold. So we subtract $13200 from the total cost of the wire mesh and we get $114000. After we added back the wire mesh, we get she will need 1500 * 30 = 4500 feet of wire mesh. We need to find the cost of the wire mesh left over, and subtract that from the total cost to find how much she needs to buy. The sol

Exception ignored in: <function InferenceSession.__del__ at 0x7f35fd372af0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/petals/client/inference_session.py", line 336, in __del__
    self.close()
  File "/usr/local/lib/python3.8/dist-packages/petals/client/inference_session.py", line 327, in close
    self._exit_server_sessions(self._server_sessions)
  File "/usr/local/lib/python3.8/dist-packages/petals/client/inference_session.py", line 207, in _exit_server_sessions
    session.__exit__(None, None, None)
  File "/usr/local/lib/python3.8/dist-packages/petals/client/inference_session.py", line 157, in __exit__
    self.close()
  File "/usr/local/lib/python3.8/dist-packages/petals/client/inference_session.py", line 134, in close
    RemoteExpertWorker.run_coroutine(self._aclose_stream())
  File "/usr/local/lib/python3.8/dist-packages/hivemind/moe/client/remote_expert_worker.py", line 47, in run_coroutine
    result = future.result()
  File "/usr/lib/p

4  None


# Эксперименты с моделью BLOOM-7b при ансамблированном режиме CoT Prompting;
Как и раньше, проходим по нескольким вариантам генерации, и повторяем эксперимент 2 раза, для того, чтобы избевиться от элемента стохастичности. Берем только первые 100 примеров из датасета, те же самые.

In [ ]:

NUM_OF_LAUNCHES = 100
NUM_REASONING_PATH = 5


kwargs = {"model_size": "mini", "is_bonus": False, "gpu": True}


for gen_regime in ["sample", "top-k", "top-p"]:
    for seed in range(1,3):

        kwargs["gen_regime"] = gen_regime
        kwargs["seed"] = seed


        model = Model(**kwargs )

        results = []
        for i in range(NUM_OF_LAUNCHES):
            example = dataset_processer.dataset[i]
            prompt_index = random.sample(list(range(8)), 1)[0]
            prompt = model.prompts[prompt_index]
            prompt_plus_question = model.prompt_plus_quesion(prompt, example["question"] )
            
            

            result = {}
            result["indx_example"] = i
            result["prompt_index"] = prompt_index

            outputs_lst = []
            
            # генерируем ответ столько, сколько reasoning paths num
            for j in range(NUM_REASONING_PATH):
                outputs = model.produce_answer(prompt_plus_question)
                
                print(i, j, outputs, output_processer.parse_num(outputs))
                outputs_lst.append(outputs)

            result["outputs"] = outputs_lst
            results.append(result)
            path_for_results = "results/results_{0}_{1}_{2}_seed_{3}.jsonl".format(kwargs["model_size"], "ansamble", gen_regime, seed)
            with open(path_for_results, 'w') as f:
                json.dump(results, f)

            os.system("cp {} /content/drive/MyDrive".format(path_for_results))


Downloading:   0%|          | 0.00/322 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/786 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.06G [00:00<?, ?B/s]

0 0  Each balloon holds 10 liters of air. So if he bought 500-balloon safes, he needed 500 x 10 = 5000 liters of gas to fill the balloons.
 5000.0
0 1  First, John bought 1000 balloons. Each balloon has 10 liters of air. Hence after buying one balloon, the total amount of air is 1000*0.10 = 10 L. Now, 1000 L is 'consumed' as we bought / expanded the first balloon. Hence before expanding the first balloon, the bank had 2000 L of air to be spent. Note that at 1000 balloons, we spend exactly 50 more tanks of gas. Hence after expanding the first 50 balloons, there were 1950 tanks of air available. Note that when we have a balloon with exactly 10 L of air, we spend exactly 50 more banks of gas to expand it to 20 L. So after expanding the first 100 balloons (we have 50 more balloons with 10 L of air and 150 balloons with 20 L of air) we already have 2000-0= 1950 banks of gas. The next cotton-ball has 30 L of air. Thus the bank of is 1000-0+0= 1950 -50, that is 1950/30, or 700 banks of air le

# Эксперименты с моделью BLOOM-7b при модифицированном ансамблированном режиме CoT Prompting;
Все также. Стоит заметить, что если все значения, которые дают промпты равны, то в качестве ответа должен рассматриваться тот, что сгененировался с промптом, наиболее семантически близким к вопросу.

In [ ]:

NUM_OF_LAUNCHES = 30
NUM_REASONING_PATH = 5

kwargs = {"model_size": "mini", "is_bonus": True, "gpu": True}

for gen_regime in ["sample", "top-k", "top-p"]:
    for seed in range(1,3):

        kwargs["gen_regime"] = gen_regime
        kwargs["seed"] = seed


        model = Model(**kwargs )

        results = []
        for i in range(NUM_OF_LAUNCHES):
            example = dataset_processer.dataset[i]


            # получаем наиоблее семантически близки к впоросу промпты
            idx_prompts = model.select_idx_prompts(example['question'])
            idx_prompts = [int(idx_prompt) for idx_prompt in idx_prompts]
            prompts = [model.prompts[idx] for idx in idx_prompts]

            result = {}
            result["indx_example"] = i
            result["prompts_index"] = idx_prompts
            outputs_lst = []

            for j in range(NUM_REASONING_PATH):

                prompt_plus_question = model.prompt_plus_quesion(prompts[j], example["question"] )
                outputs = model.produce_answer(prompt_plus_question)
                
                result["outputs"] = outputs
                print(i, j, outputs, output_processer.parse_num(outputs))
                outputs_lst.append(outputs)

            result["outputs"] = outputs_lst
            results.append(result)
            path_for_results = "results/results_{0}_{1}_{2}_seed_{3}.jsonl".format(kwargs["model_size"], "bonus", gen_regime, seed)
            with open(path_for_results, 'w') as f:
                json.dump(results, f)

            os.system("cp {} /content/drive/MyDrive".format(path_for_results))




Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0 0  Each balloon is $10. Solution is 50. So 50/1000 = 4.5.  The answer is 4 & 1/2. So 4 & 1/2/1000 = 0.4 which is the answer.
 0.4
0 1  The answer is 100 - 500 = 200.  # 100 balloons; each balloon holds 10 liters / # 100 tanks; each tank holds 10 liters ( = 1000 liters / 100 = 10 liters).
 10.0
0 2  1000 liters / 10 liter = 100. So he will need 100 tanks.</s> 100.0
0 3  John uses gas in the same way balloons use air. His 1000 balloons are like 1000 - 50 = 950 liters. Each balloon uses 10 liters of air. If he buys 500-liter tanks of gas, he has to buy (655) 500 = 1250 liters tanks of gas for his banks of balloons.
 1250.0
0 4  With his laptop (the calculator) we can do a nice calculation (see link below): 1000 * 10 = 1000 0.10. 5000 = 5000 x 10 = 5000 0.30. 1000 - 5000 = 500. And 500 is exactly the number of liters of air John can buy to fill 1000 balloons (see the calculation in the next paragraph).  
 1000.0
1 0  Mark has 12 - 3 = 9 candy bars. Now he gets two Mars bars out. The cand

In [8]:

NUM_OF_LAUNCHES = 30
NUM_REASONING_PATH = 5

kwargs = {"model_size": "mini", "is_bonus": True, "gpu": True}

for gen_regime in ["sample"]:
    for seed in range(2,3):

        kwargs["gen_regime"] = gen_regime
        kwargs["seed"] = seed


        model = Model(**kwargs )

        results = []
        for i in range(NUM_OF_LAUNCHES):
            example = dataset_processer.dataset[i]


            # получаем наиоблее семантически близки к впоросу промпты
            idx_prompts = model.select_idx_prompts(example['question'])
            idx_prompts = [int(idx_prompt) for idx_prompt in idx_prompts]
            prompts = [model.prompts[idx] for idx in idx_prompts]

            result = {}
            result["indx_example"] = i
            result["prompts_index"] = idx_prompts
            outputs_lst = []

            for j in range(NUM_REASONING_PATH):

                prompt_plus_question = model.prompt_plus_quesion(prompts[j], example["question"] )
                outputs = model.produce_answer(prompt_plus_question)
                
                result["outputs"] = outputs
                print(i, j, outputs, output_processer.parse_num(outputs))
                outputs_lst.append(outputs)

            result["outputs"] = outputs_lst
            results.append(result)
            path_for_results = "results/results_{0}_{1}_{2}_seed_{3}.jsonl".format(kwargs["model_size"], "bonus", gen_regime, seed)
            with open(path_for_results, 'w') as f:
                json.dump(results, f)

            os.system("cp {} /content/drive/MyDrive".format(path_for_results))




Downloading:   0%|          | 0.00/322 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/786 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.06G [00:00<?, ?B/s]

RuntimeError: ignored

# Добавим суффикс "The answer is" в конец ответа, и скормим опять модели

In [ ]:
import itertools

In [5]:
kwargs = {"model_size": "mini", "is_bonus": False, "gpu": True,
          "gen_regime": "sample", "seed": 1, "sec_proc": True}

model = Model(**kwargs )

mini_lst = list(itertools.product(["mini"], ["standard", "ansamble", "bonus"], ["sample", "top-k", "top-p"], [1,2]))
big_lst = list(itertools.product(["big"], ["standard"], ["sample", "top-k"], [1,2]))
for model_size, realization,gen_regime, seed in  (mini_lst + big_lst):

    path = 'results_{0}_{1}_{2}_seed_{3}.jsonl'.format(model_size, realization,
                                                          gen_regime, seed)
    with open(path, "r") as fh:
        data = json.loads(fh.read())


    for i, example in enumerate(data):
        
        example["sec_proc"] = model.sec_proc(dataset_processer, model_size, realization,
                                              gen_regime, seed, i, example)
        
        with open(path, 'w') as f:
           json.dump(data, f)

        os.system("cp {} /content/drive/MyDrive".format(path))

NameError: ignored

# Посчитаем точность предсказания вместе с погрешностями
###Для стандартной реализации CoT модели bloom 7b:

In [ ]:
acc_lst = [compute_accuracy(dataset_processer, output_processer, "mini", "standard", "sample", seed)["acc"] for seed in range(1,3)]

print("="*80 + "\nAcc: {0}% +- {1}%".format(np.mean(acc_lst) * 100 , compute_delta(acc_lst) * 100))

seed = 1

seed = 2

Question: After complaints from the residents of Tatoosh about the number of cats on the island, the wildlife service carried out a relocation mission that saw the number of cats on the island drastically reduced. On the first relocation mission, 600 cats were relocated from the island to a neighboring island. On the second mission, half of the remaining cats were relocated to a rescue center inland. If the number of cats originally on the island was 1800, how many cats remained on the island after the rescue mission?
Answer:  on the first mission that consisted of 600 cats, there were 1800 - 600 = 900 cats on the island. On the second mission that included 400 cats, there were 900 - 400 = 600…

Num_candidate = 600.0
Num_golden = 600.0
Question: Phoebe eats 1 serving and gives her dog 1 serving of peanut butter for a bedtime snack. Each jar of peanut butter has 15 servings.  How many jars will she need to make sure she and her dog have enough to last for 30 days?
An

In [4]:
acc_lst = [compute_accuracy(dataset_processer, output_processer, "mini", "standard", "top-k", seed)["acc"] for seed in range(1,3)]
print("="*80 + "\nAcc: {0}% +- {1}%".format(np.mean(acc_lst)  * 100 , compute_delta(acc_lst) * 100))

seed = 1

Question: After complaints from the residents of Tatoosh about the number of cats on the island, the wildlife service carried out a relocation mission that saw the number of cats on the island drastically reduced. On the first relocation mission, 600 cats were relocated from the island to a neighboring island. On the second mission, half of the remaining cats were relocated to a rescue center inland. If the number of cats originally on the island was 1800, how many cats remained on the island after the rescue mission?
Answer:  There were 1800 cats on the island, but after the first relocation, there were 1600 and then after the second relocation there were 600. After two relocation, the number of cats on the island decreased by 1400. Since there were 1800 cats in the first place, the number of remaining cats is 1800 - 1400 = 600</s>
Num_candidate = 600.0
Num_golden = 600.0
Question: This weekend's football game matched the Seattle Seahawks with the Denver Broncos. The final s

In [ ]:
acc_lst = [compute_accuracy(dataset_processer, output_processer, "mini", "standard", "top-p", seed)["acc"] for seed in range(1,3)]
print("="*80 + "\nAcc: {0}% +- {1}%".format(np.mean(acc_lst) * 100 , compute_delta(acc_lst) * 100))

seed = 1

Question: Renaldo drove 15 kilometers. Ernesto drove 7 kilometers more than one-third of Renaldo's distance. How many kilometers did the 2 men drive in total?
Answer:  Ernesto drove (7+15)/3 = 12.5 kilometers. Therefore, Renaldo drove 12.5 + 15 = 27 kilometers. The answer is 27.

Num_candidate = 27.0
Num_golden = 27.0
Question: Bailey is making a rectangle out of a 100cm length of rope he has. If the longer sides of the rectangle were 28cm long, what is the length of each of the shorter sides?
Answer:  The longer sides of the rectangle are 100 - 28 = 56 cm and the shorter sides are 56 - 28 = 22 cm.

Num_candidate = 22.0
Num_golden = 22.0
Question: Thomas has 25 drawings to display. 14 of the drawings were made using colored pencils. 7 of the drawings were made using blending markers. The rest of the drawings were made with charcoal. How many are charcoal drawings?
Answer:  Thomas has 25 drawings. 14 of the drawings were made using colored pencils. 7 of the drawings were made 

###Для стандартной реализации CoT модели bloom 176b:

In [ ]:
acc_lst = [compute_accuracy(dataset_processer, output_processer, "big", "standard", "sample", seed)["acc"] for seed in range(1,3)]
print("="*80 + "\nAcc: {0}% +- {1}%".format(np.mean(acc_lst) * 100 , compute_delta(acc_lst) * 100))

seed = 1

Question: Ten adults went to a ball game with eleven children. Adult tickets are $8 each and the total bill was $124. How many dollars is one child's ticket?
Answer:  The total cost would be $10$ adults times $8$ dollars + $11$ children times $7$ dollars = $ 124$. There were $11$ children who went to the ball game, so we need to figure out how much it cost for one child, $10$ adults times $8$ = $80$ + $11$ children times $7$ = $77$. So $80-76$ = $4$ dollars. So one child's ticket cost $4$ dollars.

Num_candidate = 4.0
Num_golden = 4.0
seed = 2

Question: Jessie invited 4 friends over to play. They made muffins for a snack and Jesse wants to divide them equally between herself and her friends. If they made 20 muffins in total, how many will each person have?
Answer:  The question details 20 muffins, so we can leave 20 as-is. Jessie is one person, so she has 20/4 muffins. 4 is the same as 4 friends, so the friends will have 20/4, too.


Num_candidate = 4.0
Num_golden = 4.0
Acc:

In [ ]:
acc_lst = [compute_accuracy(dataset_processer, output_processer, "big", "standard", "top-k", seed)["acc"] for seed in range(1,3)]
print("="*80 + "\nAcc: {0}% +- {1}%".format(np.mean(acc_lst) * 100 , compute_delta(acc_lst) * 100))

seed = 1

seed = 2

Question: Jessie invited 4 friends over to play. They made muffins for a snack and Jesse wants to divide them equally between herself and her friends. If they made 20 muffins in total, how many will each person have?
Answer:  The question does not give us this information. Only that Jesse wants to distribute the muffins equally. If Jessie has 4 friends, then she will have 5 muffins. 20 / 5 = 4. 

Num_candidate = 4.0
Num_golden = 4.0
Acc: 6.25% +- 6.25%


###Для ансмаблированного CoT:

In [ ]:
exp_len = 30

In [ ]:
acc_lst = [compute_accuracy(dataset_processer, output_processer, "mini", "ansamble", "sample", seed) for seed in range(1,3)]
print("="*80 + "\nAcc: {:2.2f}% +- {:2.2f}%".format(np.mean(acc_lst)  * 100 , compute_delta(acc_lst) * 100))

seed = 1



FileNotFoundError: ignored

In [ ]:
acc_lst = [compute_accuracy(dataset_processer, output_processer, "mini", "ansamble", "top-k", seed) for seed in range(1,3)]
print("="*80 + "\nAcc: {:2.2f}% +- {:2.2f}%".format(np.mean(acc_lst)  * 100 , compute_delta(acc_lst) * 100))

In [ ]:
analysis_lst = [compute_accuracy(dataset_processer, output_processer, "mini", "ansamble", "top-p", seed) for seed in range(1,3)]
acc_lst, cnt_g_in_cand, cnt_ans_w = ([analysis[key] for analysis in analysis_lst] for key in ["acc", "cnt_golden_in_candidates", "cnt_ansamble_works"])
print("="*80 + "\nAcc: {:2.2f}% +- {:2.2f}%".format(np.mean(acc_lst)  * 100 , compute_delta(acc_lst) * 100))

seed = 1

Question: Mark has 12 candy bars in total between Mars bars, Snickers, and Butterfingers.  He has 3 Snickers and 2 Mars bars.  How many Butterfingers does he have?
Answer: [' If he has 3 Snickers and 2 Mars bars, then he is left with 7 candy bars. This is done with this equation.\n', ' Mark has 3 Snickers, 2 Mars bars, and 6 Butterfingers. 12 – (3 + 2) = 7. The answer is 7.\n', ' Mark starts with 12 Mars bars. If he got 3 Snickers and 2 Mars bars, then that is 5 more Mars bars. 12 + 5 = 17. The answer is 17.\n', ' Mark has 12 candy bars. He has 2 Mars bars and 2 Butterfingers. Therefore, the number of Butterfingers is 4.\n', ' Mark starts with 12 candy bars. From the first candy bar, he gets 3 candy bars each of Mars bars and Snickers.  In total, he has 2 candy bars of each type.  He has 3 Snickers and 2 Mars bars, for a total of 4 candy bars.  4 + 12 = 18.\n']
Num_candidate = 7.0
Num_golden = 7.0
Question: Shane prepares sandwiches for the Boy Scouts. He buys 2 packages of s

In [ ]:
print("golden in candidates: {:2.2f}%\nansamble works: {:2.2f}%".format(np.mean(cnt_g_in_cand) / exp_len * 100, np.mean(cnt_ans_w) / exp_len * 100))

golden in candidates: 8.3333%
ansamble works: 21.6667%


In [ ]:
mn = np.mean([compute_accuracy(dataset_processer, \
                          output_processer, \
                          "mini", "ansamble", gen_regime, seed)["cnt_golden_in_candidates"] \
         for seed in range(1,3) for gen_regime in ["sample", "top-k", "top-p"]])
"cnt_golden_in_candidates_mean = {} %".format(mn*100)

[None, None, None, None, None, None]

Посчитаем, как часто ансамбль работает, то есть как часто в ответах - кандитатах содержатся хотя бы раз совпадающие ответы?

In [ ]:
mn = np.mean([compute_accuracy(dataset_processer, \
                          output_processer, \
                          "mini", "ansamble", gen_regime, seed)["cnt_ansamble_works"] \
         for seed in range(1,3) for gen_regime in ["sample", "top-k", "top-p"]])
"cnt_golden_in_candidates_mean = {} %".format(mn*100)

###Для ансмаблированного модифицированного CoT (bonus):

In [ ]:
acc_lst = [compute_accuracy(dataset_processer, output_processer, "mini", "bonus", "sample", seed) for seed in range(1,3)]
print("="*80 + "\nAcc: {0}% +- {1}%".format(np.mean(acc_lst)  * 100 , compute_delta(acc_lst) * 100))

In [ ]:
acc_lst = [compute_accuracy(dataset_processer, output_processer, "mini", "bonus", "top-k", seed) for seed in range(1,3)]
print("="*80 + "\nAcc: {0}% +- {1}%".format(np.mean(acc_lst)  * 100 , compute_delta(acc_lst) * 100))

In [ ]:
acc_lst = [compute_accuracy(dataset_processer, output_processer, "mini", "bonus", "top-p", seed) for seed in range(1,3)]
print("="*80 + "\nAcc: {0}% +- {1}%".format(np.mean(acc_lst)  * 100 , compute_delta(acc_lst) * 100))